In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds

tf.__version__

'2.10.0'

In [3]:
(ds_train, ds_validation), ds_info = tfds.load(
      name='beans',
      split=['train', 'validation'],
      as_supervised=True,
      with_info=True)

2023-05-22 22:32:19.637655: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1034 [00:00<?, ? examples/s]

...

Generating validation examples...:   0%|          | 0/133 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/beans/0.1.0.incompleteTW90DC/beans-validation.tfrecord*...:   0%|          | 0…

Generating test examples...:   0%|          | 0/128 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/beans/0.1.0.incompleteTW90DC/beans-test.tfrecord*...:   0%|          | 0/128 […

Dataset beans downloaded and prepared to ~/tensorflow_datasets/beans/0.1.0. Subsequent calls will reuse this data.


In [4]:
def preprocess(image, label):
  resized_img = tf.image.resize(image, [300, 300])
  resized_img = resized_img / 255.
  return resized_img, label

BATCH_SIZE = 32

In [5]:
ds_train = ds_train.map(preprocess).cache().shuffle(
    ds_info.splits['train'].num_examples).batch(BATCH_SIZE).prefetch(
    tf.data.experimental.AUTOTUNE)
ds_validation = ds_validation.map(preprocess).batch(BATCH_SIZE).cache(

).prefetch(tf.data.experimental.AUTOTUNE)

In [16]:
model = tf.keras.models.Sequential([

        # ADD LAYERS OF THE MODEL HERE
        tf.keras.layers.Conv2D(10, 3, activation="relu", input_shape=(300, 300, 3)),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(10, 3, activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(10, 3, activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Flatten(),
        # If you don't adhere to the instructions in the following comments,
        # tests will fail to grade your model:
        # The input layer of your model must have an input shape of
        # (300,300,3).
        # Make sure that your last layer has 3 (number of classes) neurons
        # activated by softmax.
        tf.keras.layers.Dense(3, activation='softmax')
    ])

In [17]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [18]:
earlyStoppingCallback = tf.keras.callbacks.EarlyStopping(
      monitor="val_loss",
      patience=200,
      restore_best_weights=True)

In [19]:
history_10 = model.fit(ds_train,
                       epochs=20,
                       steps_per_epoch=len(ds_train),
                       validation_data=ds_validation,
                       validation_steps=len(ds_validation),
                       callbacks=[earlyStoppingCallback])

Epoch 1/20
33/33 [==============================] - 6s 172ms/step - loss: 1.0021 - accuracy: 0.5019 - val_loss: 0.8773 - val_accuracy: 0.5865
Epoch 2/20
33/33 [==============================] - 6s 170ms/step - loss: 0.7956 - accuracy: 0.6364 - val_loss: 0.7809 - val_accuracy: 0.6391
Epoch 3/20
33/33 [==============================] - 6s 170ms/step - loss: 0.6786 - accuracy: 0.7099 - val_loss: 0.6676 - val_accuracy: 0.6617
Epoch 4/20
33/33 [==============================] - 6s 171ms/step - loss: 0.5873 - accuracy: 0.7611 - val_loss: 0.6013 - val_accuracy: 0.7669
Epoch 5/20
33/33 [==============================] - 6s 171ms/step - loss: 0.5445 - accuracy: 0.7882 - val_loss: 0.6131 - val_accuracy: 0.7218
Epoch 6/20
33/33 [==============================] - 6s 171ms/step - loss: 0.4741 - accuracy: 0.8133 - val_loss: 0.5210 - val_accuracy: 0.7820
Epoch 7/20
33/33 [==============================] - 6s 171ms/step - loss: 0.3951 - accuracy: 0.8549 - val_loss: 1.0075 - val_accuracy: 0.5564
Epoch 

In [20]:
model.evaluate(ds_validation)

5/5 [==============================] - 0s 33ms/step - loss: 0.6623 - accuracy: 0.7895


[0.6622675061225891, 0.7894737124443054]